In [21]:
import pandas as pd 
import requests
from bs4 import BeautifulSoup

# Getting real estate listings


## Zumber

In [114]:
def Zumber(content): 
    soup = BeautifulSoup(content, "html.parser")
    rent = [x.text for x in soup.find_all("div", class_="css-uzc059")]
    bath = [x.text for x in soup.find_all("div", class_="css-1iq6kk8")]


    dict = {'Monthly': rent, 'Number of baths': bath}
    df= pd.DataFrame(dict)
    df['Monthly'] = dfOneBed['Monthly'].str.replace('\D+', '')
    return df

### Studios

In [115]:
URL = "https://www.zumper.com/apartments-for-rent/vancouver-bc/west-end/studios"
requestURL = requests.get(URL)
content = requestURL.content

In [116]:
Zumber(content)

C:\Users\Jimmy\AppData\Local\Temp/ipykernel_18148/605094714.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Monthly'] = dfOneBed['Monthly'].str.replace('\D+', '')


,Monthly,Number of baths
0,3875,Studio • 1 Bath
1,3150,Studio • 1 Bath
2,2680,Studio • 1 Bath
3,2750,Studio • 1 Bath
4,2614,Studio • 1 Bath
5,3000,Studio • 1 Bath
6,2629,Studio • 1 Bath
7,3700,Studio • 1 Bath
8,2706,Studio • 1 Bath
9,2743,Studio • 1 Bath


### 1 bed

In [117]:
URL2 = "https://www.zumper.com/apartments-for-rent/vancouver-bc/west-end/1-beds"
requestURL2 = requests.get(URL2)
content2 = requestURL2.content

In [118]:
Zumber(content2)

C:\Users\Jimmy\AppData\Local\Temp/ipykernel_18148/605094714.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Monthly'] = dfOneBed['Monthly'].str.replace('\D+', '')


,Monthly,Number of baths
0,3875,1 Bed • 1 Bath
1,3150,1 Bed • 1 Bath
2,2680,1 Bed • 1 Bath
3,2750,1 Bed • 1 Bath
4,2614,1 Bed • 1 Bath
5,3000,1 Bed • 1 Bath
6,2629,1 Bed • 1 Bath
7,3700,1 Bed • 1 Bath
8,2706,1 Bed • 1 Bath
9,2743,1 Bed • 1 Bath


$## 2 Beds

In [119]:
URL3 = "https://www.zumper.com/apartments-for-rent/vancouver-bc/west-end/2-beds"
requestURL3 = requests.get(URL3)
content3 = requestURL3.content


In [120]:
Zumber(content3)

C:\Users\Jimmy\AppData\Local\Temp/ipykernel_18148/605094714.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Monthly'] = dfOneBed['Monthly'].str.replace('\D+', '')


,Monthly,Number of baths
0,3875,2 Beds • 1 - 2 Baths
1,3150,2 Beds • 1 - 2 Baths
2,2680,2 Beds • 2 Baths
3,2750,2 Beds • 2 Baths
4,2614,2 Beds • 1 - 2 Baths
5,3000,2 Beds • 1 - 2 Baths
6,2629,2 Beds • 2 Baths
7,3700,2 Beds • 1 Bath
8,2706,2 Beds • 2 Baths
9,2743,2 Beds • 2 Baths


### 3 beds

In [122]:
URL4 = "https://www.zumper.com/apartments-for-rent/vancouver-bc/west-end/3-beds"
requestURL4 = requests.get(URL4)
content4 = requestURL4.content


In [123]:
Zumber(content4)

C:\Users\Jimmy\AppData\Local\Temp/ipykernel_18148/605094714.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Monthly'] = dfOneBed['Monthly'].str.replace('\D+', '')


,Monthly,Number of baths
0,3875,3 Beds • 2 Baths
1,3150,3 Beds • 2 Baths
2,2680,3 Beds • 3 Baths
3,2750,3 Beds • 2 Baths
4,2614,3 Beds • 2 Baths
5,3000,3 Beds • 2 Baths
6,2629,3 Beds • 2 Baths
7,3700,3 Beds • 2 Baths
8,2706,3 Beds • 1 Bath
9,2743,3 Beds • 2 Baths


## Kijiji

In [92]:
def kijiji(content): 
    soup = BeautifulSoup(content, "html.parser")
    realEstate = soup.findAll("div", class_="container-results large-images")[1]
    realEstate2 = [x.find('div', class_="info") for x in realEstate.findAll("div", attrs={"data-listing-id":True})]

    realEstateLinks = [x.find('div', class_=["title"]) for x in realEstate2]
    realEstateLinks1 = [x.find('a', attrs={"href":True}) for x in realEstate2]
    realEstateLinks = ["https://www.kijiji.ca"+ x['href'] for x in realEstateLinks1]

    dfKijiji = pd.DataFrame(columns=['Monthly', 'Number of baths'])

    for i in range(len(realEstateLinks)):
        requestsIterate = requests.get(realEstateLinks[i])
        soup3 = BeautifulSoup(requestsIterate.text, "html.parser")


        for x in soup3.find_all('div', class_=['realEstateTitle-1440881021']):
            monthlyNum = x.find('span')['content']
            bathroomNum = x.findAll('span', class_="noLabelValue-3861810455")[-1].text

            dfKijiji = dfKijiji.append({'Monthly': monthlyNum, 'Number of baths': bathroomNum}, ignore_index=True)
    return dfKijiji
    



### Studios

In [31]:
URL2 = "https://www.kijiji.ca/b-apartments-condos/vancouver-downtown-west-end/bachelor+studio/c37l1700292a27949001"
requestURL2 = requests.get(URL2)
content2 = requestURL2.content

soup2 = BeautifulSoup(content2, "html.parser")




In [93]:
kijiji(content2)

,Monthly,Number of baths
0,2369.,Bathrooms: 1
1,2650.,Bathrooms: 1
2,1640.,Bathrooms: 1
3,2369.,Bathrooms: 1
4,2195.,Bathrooms: 1
5,2375.,Bathrooms: 1
6,2325.,Bathrooms: 1
7,2900.,Bathrooms: 1
8,2268.,Bathrooms: 1
9,2570.,Bathrooms: 1


### 1 bedroom

In [94]:
URL2 = "https://www.kijiji.ca/b-apartments-condos/vancouver-downtown-west-end/1+bedroom/c37l1700292a27949001"
requestURL2 = requests.get(URL2)
contentOneBedRoom = requestURL2.content

In [95]:
kijiji(contentOneBedRoom)

,Monthly,Number of baths
0,2600.,Bathrooms: 1
1,2599.,Bathrooms: 1
2,2700.,Bathrooms: 1
3,2600.,Bathrooms: 1
4,2750.,Bathrooms: 1
5,2300.,Bathrooms: 1
6,2532.,Bathrooms: 1
7,2629.,Bathrooms: 1
8,2500.,Bathrooms: 1
9,2550.,Bathrooms: 1


### 2 bed

In [96]:
URL2 = "https://www.kijiji.ca/b-apartments-condos/vancouver-downtown-west-end/2+bedroom/c37l1700292a27949001"
requestURL2 = requests.get(URL2)
contentTwoBedRoom = requestURL2.content

In [97]:
kijiji(contentTwoBedRoom)

,Monthly,Number of baths
0,3599.,Bathrooms: 1
1,2600.,Bathrooms: 1
2,2369.,Bathrooms: 1
3,5800.,Bathrooms: 2
4,5975.,Bathrooms: 2
5,4500.,Bathrooms: 2
6,6550.,Bathrooms: 2
7,3875.,Bathrooms: 1
8,6680.,Bathrooms: 3
9,9000.,Bathrooms: 5.5


### 3 bed

In [98]:
URL2 = "https://www.kijiji.ca/b-apartments-condos/vancouver-downtown-west-end/3+bedroom/c37l1700292a27949001"
requestURL2 = requests.get(URL2)
contentThreeBedRoom = requestURL2.content

In [99]:
kijiji(contentThreeBedRoom)

,Monthly,Number of baths
0,3599.,Bathrooms: 1
1,2600.,Bathrooms: 1
2,2369.,Bathrooms: 1
3,5800.,Bathrooms: 2
4,5975.,Bathrooms: 2
5,4500.,Bathrooms: 2
6,6550.,Bathrooms: 2
7,3875.,Bathrooms: 1
8,6680.,Bathrooms: 3
9,9000.,Bathrooms: 5.5


## Pad Mapper

In [86]:
def padMapper(content): 
    soupPad = BeautifulSoup(content, "html.parser").find('body')

    realEstate = soupPad.findAll("div", class_="ListItemMobile_text__3KFu_")
    realEstate2 = [x.text.strip() for x in realEstate]
    return realEstate2

### Studio 

In [83]:
URL3 = "https://www.padmapper.com/apartments/vancouver-bc/west-end/studios"
requestURL3 = requests.get(URL3)
content3 = requestURL3.content

In [88]:
padMapper(content3)

['$1,900+',
 '$2,369',
 '$2,495',
 '$2,700+',
 '$2,200',
 '$2,268',
 '$2,375',
 '$2,500',
 '$2,095',
 '$2,000',
 '$2,570',
 '$2,650',
 '$2,450+',
 '$1,625',
 '$2,650',
 '$3,500']

### 1 bathrooms

In [89]:
URL4 = "https://www.padmapper.com/apartments/vancouver-bc/west-end/1-beds"
requestURL4 = requests.get(URL4)
content4 = requestURL4.content

padMapper(content4)

['$2,680+',
 '$2,450+',
 '$3,875',
 '$3,150+',
 '$2,355+',
 '$1,800+',
 '$2,325+',
 '$2,629',
 '$2,889',
 '$2,706',
 '$2,743',
 '$2,600',
 '$2,850',
 '$2,550',
 '$2,400',
 '$2,750+',
 '$2,614',
 '$2,695',
 '$2,532',
 '$2,361',
 '$2,550',
 '$2,375+',
 '$2,395']

### 2 bathrooms

In [90]:
URL4 = "https://www.padmapper.com/apartments/vancouver-bc/west-end/2-beds"
requestURL4 = requests.get(URL4)
content4 = requestURL4.content

padMapper(content4)

['$4,500+',
 '$4,300+',
 '$4,100+',
 '$3,495',
 '$4,900',
 '$3,599',
 '$3,799',
 '$3,895',
 '$4,850',
 '$3,428',
 '$3,600+',
 '$3,300+',
 '$3,262',
 '$3,650+',
 '$3,400',
 '$3,450',
 '$2,650',
 '$3,050',
 '$3,900',
 '$2,950',
 '$7,800',
 '$7,500',
 '$11K']

In [91]:
URL4 = "https://www.padmapper.com/apartments/vancouver-bc/west-end/3-beds"
requestURL4 = requests.get(URL4)
content4 = requestURL4.content

padMapper(content4)

['$6,550',
 '$4,500',
 '$5,195',
 '$6,200',
 '$5,000',
 '$4,500+',
 '$4,500',
 '$12.9K',
 '$4,900',
 '$4,300']